In [1]:
from openai import AzureOpenAI
import os

from dotenv import load_dotenv
### ONLY FOR LOCAL TEST ###
dotenv_path = ".env"
load_dotenv(dotenv_path)

client = AzureOpenAI()

index_name = "course-questions"

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[947]

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

In [4]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

In [20]:
from tqdm.auto import tqdm

/Users/naibahom/miniconda3/envs/llm-training/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

100%|██████████| 948/948 [00:00<00:00, 1409.24it/s]


In [31]:
def elastic_search(query, index_name):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        print(hit['_score'])
    
    return result_docs

In [36]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [42]:
import tiktoken

def calc_token(string: str) -> int:
    """Returns the number of tokens in a text string."""
    enc = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(enc.encode(string))
    return num_tokens
    

In [37]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [45]:
def rag(query):
    search_results = elastic_search(query, index_name=index_name)
    prompt = build_prompt(query, search_results)
    print(f"Prompt length: {len(prompt)}")
    token_length = calc_token(string=prompt)
    print(f"Token: {token_length}")
    print(prompt)
    answer = llm(prompt)
    return answer

In [46]:
query = 'How do I execute a command in a running docker container?'

rag(query)

84.050095
51.04628
49.938507
Prompt length: 1499
Token: 323
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT: 
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The

'To execute a command in a running Docker container, follow these steps:\n\n1. First, find the container ID of the running container:\n   ```sh\n   docker ps\n   ```\n\n2. Once you have the container ID, you can execute a command in that specific container. For example, to start a bash shell inside the container, use:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of your running container.'